In [2]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [13]:
#MODEL
list_i = []
list_return_price = []
list_pcapercent = []
list_rmse, list_mse, list_mae = [], [], []
list_k = []
list_t = []
list_r2_train = []
list_r2_test = []

scaler = StandardScaler()
pca = PCA(n_components = 20)

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    a = data.loc[:, ~data.columns.isin(['Today'])]

    a_scale = scaler.fit_transform(a)
    a_scale_df = pd.DataFrame(a_scale, columns=a.columns)
    pF = pca.fit_transform(a_scale_df)
    pF_df = pd.DataFrame(data = pF) 
    ss = pca.explained_variance_ratio_
    total = sum(ss)
    list_pcapercent.append(total)
    b = pd.DataFrame()
    b['Today'] = data['Today'].reset_index(drop=True)
    pF_final = pd.concat([pF_df, b], axis=1)
    
    X = pF_final.loc[:, ~pF_final.columns.isin(['Today'])]
    y = pF_final['Today']

    list_k1 = []
    list_t1 = []
    list_score = []

    x = round(len(X)*0.1)
    for t in range(2,x+1):
        for k in np.arange(0.1, 0.5, 0.01):
            model = KNeighborsRegressor(n_neighbors=t)  
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=k, random_state=12, shuffle=False)
            model.fit(X_train, y_train)
            yhat_test = model.predict(X_test)
            score = model.score(X_test, y_test)
            list_k1.append(k)
            list_t1.append(t)
            list_score.append(score)

        score_max = max(list_score)
        a = list_score.index(score_max)
        k = list_k1[a]
        t = list_t1[a]

        X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=k,random_state=12, shuffle=False)
        model.fit(X_train,y_train)
        yhat_test = model.predict(X_test)
        r2_train = model.score(X_train, y_train)
        r2_test = model.score(X_test, y_test)
        mse = mean_squared_error(y_test, yhat_test, squared=True)
        rmse = mean_squared_error(y_test, yhat_test, squared=False)
        mae = mean_absolute_error(y_test, yhat_test)

        X_predict = [X.iloc[-1].values.tolist()]
        Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_k.append(k)
    list_t.append(t)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {rmse:.2f}',f' MSE: {mse:.2f}', f'MAE: {mae:.2f}',f' test size: {k:.2f}',f'neighbors: {t}', f'pca: {total:.2f}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -0.25  R2_train: 0.22  R2_test: 0.30  RMSE: 1.61  MSE: 2.58 MAE: 1.14  test size: 0.14 neighbors: 17 pca: 0.91
Stock: BCM  Predict: 0.10  R2_train: 0.13  R2_test: 0.15  RMSE: 1.25  MSE: 1.55 MAE: 0.81  test size: 0.11 neighbors: 6 pca: 0.90
Stock: VHM  Predict: 0.22  R2_train: 0.19  R2_test: 0.20  RMSE: 1.82  MSE: 3.31 MAE: 1.28  test size: 0.14 neighbors: 26 pca: 0.90
Stock: VIC  Predict: 0.19  R2_train: 0.17  R2_test: 0.15  RMSE: 1.97  MSE: 3.88 MAE: 1.14  test size: 0.23 neighbors: 15 pca: 0.90
Stock: VRE  Predict: -0.21  R2_train: 0.19  R2_test: 0.34  RMSE: 1.28  MSE: 1.63 MAE: 0.90  test size: 0.12 neighbors: 13 pca: 0.91
Stock: BVH  Predict: 0.55  R2_train: 0.22  R2_test: 0.43  RMSE: 0.88  MSE: 0.78 MAE: 0.59  test size: 0.13 neighbors: 15 pca: 0.90
Stock: POW  Predict: 0.44  R2_train: 0.20  R2_test: 0.34  RMSE: 1.16  MSE: 1.34 MAE: 0.78  test size: 0.11 neighbors: 12 pca: 0.90
Stock: GAS  Predict: 0.38  R2_train: 0.20  R2_test: 0.43  RMSE: 0.82  MSE: 0.67 MA

In [5]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test,
                    'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,'test size': list_k,'neighbors': list_t, 'pca': list_pcapercent})
KQ.to_excel("Kết quả sau PCA.xlsx")
KQ.head(42069)

NameError: name 'list_i' is not defined